In [49]:
!pip install transformers
!pip install datasets #dasets de huggin face

#IMPORTANTE trabajar con las librerias actualizadas o que sean compatibles con el codigo
!pip install --upgrade datasets
!pip install --upgrade transformers


In [ ]:
import datasets
import transformers

In [ ]:
#es el dataloader para descargar el datasets
from datasets import load_dataset
reviews = load_dataset('rotten_tomatoes')
reviews

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
#¿como pasamos el dataset de huggin Face a pandas?
df_train = reviews['train'].to_pandas()
df_train

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,"interminably bleak , to say nothing of boring .",0


In [ ]:
# @title score

from matplotlib import pyplot as plt
import seaborn as sns
df_train.groupby('score').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

KeyError: 'score'

In [ ]:
#PIPELINE es la manera principal de interactuar con los transformadores
from transformers import pipeline
#creamos un clasificador y ñe decimos que es un clasificador de sentimientos
#dentro de pipeline podemos decirle que tipo de MODELO vamos a utilizar
#en el ejemplo, como no especificamos el modelos, se utilizara no por defecto
clasificador = pipeline('sentiment-analysis')
clasificador('I love you')

In [ ]:
#ejemplo de como aplicar el clasificador al dataset
def score(review_text):
  return clasificador(review_text)[0]['label']

#aplicamos la funcion Score a la columna text y agregamos el resultado como una nueva columna
df_train['score'] = df_train['text'].apply(score)
df_train

In [ ]:
#para logearnos a huggin face desde la notebook
from huggingface_hub import notebook_login

notebook_login()

# NPL Pipelines

In [ ]:
help(pipeline)
#las pipeline son sencillas de usar, eliges el modelo y la tarea que quieres que realize
#con help vemos el listado de las posibles tareas que acepta pipeline


NameError: name 'pipeline' is not defined

# Tokenizacion

In [ ]:
from transformers import AutoTokenizer
#un AutoTokenaizer necesita leer un tokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')
#los LLM tienen un tokenizador dentro de ellos

In [ ]:
#tenemos un texto para tokenizar
oracion = "Preposterous, I'm flabbergasted!"

input_ids = tokenizer(oracion, return_tensors = 'pt').input_ids
#Return_tensors = 'pt' quiere decir que el tokenizador debe devolver los token como tenzores de Torch

input_ids

#Palabra --> tokens --> ids unicos --> vertor embed



tensor([[37534,  6197,   516,    11,   314,  1101,   781,   397,  3900,  8992,
             0]])

como convertimos los tokens a texto?

In [ ]:
tokenizer.decode(37534)

'Prep'

In [ ]:
for token_id in input_ids[0]:
  print(tokenizer.decode(token_id))

Prep
oster
ous
,
 I
'm
 fl
ab
berg
asted
!


si quisieramos un modelo para que complete la oracion

In [ ]:
from transformers import AutoModelForCausalLM
#utilizamos AotuModel para no recordar cada modelo con su respectivo tokenizer

gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')
texto = "I skip across the"
input_ids = tokenizer(texto, return_tensors = 'pt').input_ids
#Return_tensors = 'pt' quiere decir que el tokenizador debe devolver los token como tenzores de Torch

input_ids

#Palabra --> tokens --> ids unicos --> vertor embed

tensor([[   40, 14267,  1973,   262]])

In [ ]:
outputs = gpt2(input_ids)
outputs.logits.shape
#logits es como un softmax
#vamos a ver que torch.Size([1, 4, 50257])
#1 es el número de lote que
#4 es el número de tokens
#50257 es el tmaño total del vocavulario gpt2

torch.Size([1, 4, 50257])

In [ ]:
outputs.logits
#Logits son los resultados de brutos de los modelos, la idea es seleccionar el token con más probabilidad de salida
# la forma en que lo hacemos es que vamos a centrarnos en los logits de toda la frase de entrada y luego predecir
# la siguiente palabra.
#Notar que esta linea nos muetra cuatro filas, que corresponden a cuatro predicciones de cada una de las palabras de la oracion

tensor([[[-39.3084, -39.0100, -41.8374,  ..., -46.9337, -44.9074, -39.5149],
         [-78.0579, -78.4766, -82.9907,  ..., -87.3913, -86.9834, -79.6007],
         [-73.0664, -74.0119, -78.5556,  ..., -81.0934, -82.5249, -75.8570],
         [-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
#como nos interesa solamente la ultima palabra de la oración la seleccionamos en la sigueinte linea
final_logits = gpt2(input_ids).logits[0,-1]

In [ ]:
final_logits
#esta lista representa al vocabulario correspondente al modelo
#y cada numero que vemos está asiciado a uno de los idsTOken de ese vocabulario, por lo tanto el número mpas alto es que tiene más posisbilidades de salir

tensor([-87.5486, -86.0567, -88.3320,  ..., -88.2509, -90.5480, -86.1209],
       grad_fn=<SelectBackward0>)

In [ ]:
#utilizamos argMax para seleccionar la que tiene probabilidad más alta
final_logits.argmax() #Token ID <--> Index Location logits


tensor(1627)

In [ ]:
tokenizer.decode(final_logits.argmax())

' line'

In [ ]:
# si quisieramos ver, no solamente ver el de más probabilidad, si quisieramos ver un top de 10
import torch
top10_logits = torch.topk(final_logits, 10)

for index in top10_logits.indices:
  print(tokenizer.decode(index))


 line
 street
 river
 room
 pond
 bridge
 border
 country
 road
 board


In [ ]:
final_logits.softmax(dim=0)

tensor([5.5734e-07, 2.4776e-06, 2.5463e-07,  ..., 2.7614e-07, 2.7765e-08,
        2.3236e-06], grad_fn=<SoftmaxBackward0>)

In [ ]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)

In [ ]:
for value, index in zip(top10.values, top10.indices):
  #:.1% formatea el valor despues punto hay que decirle cuantos decimales queresmos luego de la coma
  print(f"{tokenizer.decode(index)} --{value.item():.1%}")

 line --3.6%
 street --2.7%
 river --2.2%
 room --1.9%
 pond --1.8%
 bridge --1.7%
 border --1.7%
 country --1.7%
 road --1.6%
 board --0.9%


Generar texto

In [58]:
output_ids = gpt2.generate(input_ids, max_new_tokens=20, repetition_penalty=1.2)
#siempre hay que tener encuenta aquí que, especialmente en los modelos pequeños, es que tienden a repetirse mucho.
#se puede intentar añadir una penalizacion por la repeticion

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [59]:
decoded_text = tokenizer.decode(output_ids[0])
print(decoded_text)



I skip across the line to a place where I can see it.

The next time you're in town,
